In [ ]:
import sys

sys.path.append("..")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import h5py
import timm
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from timm import create_model
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from transformers.optimization import (
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
)
import torch.nn.functional as F
from pathlib import Path
from torch.utils.data import DataLoader
import os
from fastprogress.fastprogress import master_bar, progress_bar
from fastai.vision.all import L, unsqueeze
from timm.data.mixup import Mixup
from timm.loss import (
    LabelSmoothingCrossEntropy,
    BinaryCrossEntropy,
    SoftTargetCrossEntropy,
)

from einops import rearrange

In [ ]:
class RollTimeFreq:
    def __init__(self, shift_t_percent=0.3, shift_f_percent=0.3):
        self.shift_t_percent = shift_t_percent
        self.shift_f_percent = shift_f_percent

    def __call__(self, x):
        c, t, f = x.shape
        roll_t = np.random.randint(0, int(self.shift_t_percent * t))
        roll_f = np.random.randint(0, int(self.shift_f_percent * f))
        if np.random.rand() > 0.5:
            x = np.roll(x, roll_t, 1)
        else:
            x = np.roll(x, roll_f, 2)

        if np.random.rand() > 0.5:
            k = [1, 0]
            x = x[k, ...]
        if np.random.rand() > 0.5:
            x = np.flip(x, axis=1).copy()
        if np.random.rand() > 0.5:  # vertical flip
            x = np.flip(x, axis=2).copy()
        return x

In [ ]:
def get_snr(left, right, df):
    df_ = pd.concat([df.query(f"snr>{left} & snr<{right}"), df.query("snr==0")])
    return df_


def generate_report(df, p, fn):
    pred = F.softmax(p).cpu().numpy()[:, 1]
    val_df_eval = df.copy()
    val_df_eval["pred"] = pred
    val_df_eval.to_csv(f'{fn}_oof.csv')

    roc_100 = roc_auc_score(val_df_eval["target"], val_df_eval["pred"])
    roc_50_100 = roc_auc_score(
        get_snr(50, 100, val_df_eval)["target"], get_snr(50, 100, val_df_eval)["pred"]
    )
    roc_0_50 = roc_auc_score(
        get_snr(0, 50, val_df_eval)["target"], get_snr(0, 50, val_df_eval)["pred"]
    )
    roc_0_40 = roc_auc_score(
        get_snr(0, 40, val_df_eval)["target"], get_snr(0, 40, val_df_eval)["pred"]
    )
    roc_0_30 = roc_auc_score(
        get_snr(0, 30, val_df_eval)["target"], get_snr(0, 30, val_df_eval)["pred"]
    )

    roc_0_20 = roc_auc_score(
        get_snr(0, 20, val_df_eval)["target"], get_snr(0, 20, val_df_eval)["pred"]
    )
    


    return {
        "roc_all": roc_100,
        "roc_50_100": roc_50_100,
        "roc_0_50": roc_0_50,
        "roc_0_40": roc_0_40,
        "roc_0_30": roc_0_30,
        "roc_0_20": roc_0_20,
    }

In [ ]:
class SaveModel:
    def __init__(self, folder, exp_name, best=np.inf):
        self.best = best
        self.folder = Path(folder) / f"{exp_name}.pth"

    def __call__(self, score, model, epoch):
        if score < self.best:
            self.best = score
            print(f"Better model found at epoch {epoch} with value: {self.best}.")
            torch.save(model.state_dict(), self.folder)


class SaveModelMetric:
    def __init__(self, folder, exp_name, best=-np.inf):
        self.best = best
        self.folder = Path(folder) / f"{exp_name}.pth"

    def __call__(self, score, model, epoch):
        if score > self.best:
            self.best = score
            print(f"Better model found at epoch {epoch} with value: {self.best}.")
            torch.save(model.state_dict(), self.folder)


class SaveModelEpoch:
    def __init__(self, folder, exp_name, best=-np.inf):
        self.best = best
        self.folder = Path(folder)
        self.exp_name = exp_name

    def __call__(self, score, model, epoch):
        self.best = score
        print(f"Better model found at epoch {epoch} with value: {self.best}.")
        torch.save(model.state_dict(), f"{self.folder/self.exp_name}_{epoch}.pth")


def custom_auc_score(p, gt):
    return roc_auc_score(gt.cpu().numpy(), F.softmax(p).cpu().numpy()[:, 1])


def fit_mixup(
    epochs,
    model,
    train_dl,
    valid_dl,
    loss_fn,
    opt,
    metric,
    val_df,
    folder="models",
    exp_name="exp_00",
    device=None,
    sched=None,
    mixup_=False,
    save_md=SaveModelEpoch,
):
    if device is None:
        device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )

    os.makedirs(folder, exist_ok=True)
    loss_fn_trn = loss_fn
    if mixup_:
        mixup = Mixup(
            num_classes=2, mixup_alpha=0.4, prob=0.8
        )
        loss_fn_trn = BinaryCrossEntropy()
    mb = master_bar(range(epochs))
    
   
    mb.write(["epoch", 
              "train_loss",
              "valid_loss",
              "val_metric",
              "roc_all",
              "roc_50_100", 
              "roc_0_50", 
              "roc_0_40", 
              "roc_0_30", 
              "roc_0_20"], table=True)
    model.to(device)  # we have to put our model on gpu
    scaler = torch.cuda.amp.GradScaler()  # this for half precision training
    save_md = save_md(folder, exp_name)

    for i in mb:  # iterating  epoch
        trn_loss, val_loss = 0.0, 0.0
        trn_n, val_n = len(train_dl.dataset), len(valid_dl.dataset)
        model.train()  # set model for training
        for (xb, yb) in progress_bar(train_dl, parent=mb):
            xb, yb = xb.to(device), yb.to(device)  # putting batches to device
            if mixup_:
                xb, yb = mixup(xb, yb)
            with torch.cuda.amp.autocast():  # half precision
                out = model(xb)  # forward pass
                loss = loss_fn_trn(out, yb)  # calulation loss

            trn_loss += loss.item()

            scaler.scale(loss).backward()  # backward
            scaler.step(opt)  # optimzers step
            scaler.update()  # for half precision
            opt.zero_grad()  # zeroing optimizer
            if sched is not None:
                sched.step()  # scuedular step

        trn_loss /= mb.child.total

        # putting model in eval mode
        model.eval()
        gt = []
        pred = []
        # after epooch is done we can run a validation dataloder and see how are doing
        with torch.no_grad():
            for (xb, yb) in progress_bar(valid_dl, parent=mb):
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = loss_fn(out, yb)
                val_loss += loss.item()

                gt.append(yb.detach())
                pred.append(out.detach())
        # calculating metric
        metric_ = metric(torch.cat(pred), torch.cat(gt))
        # saving model if necessary
        save_md(metric_, model, i)
        val_loss /= mb.child.total
        dict_res = generate_report(val_df, torch.cat(pred), f"{folder}/{exp_name}_{i}")
            
        pd.DataFrame(
            {
                "trn_loss": [trn_loss],
                "val_loss": [val_loss],
                "metric": [metric_],
                "roc_all": [dict_res["roc_all"]],
                "roc_50_100": [dict_res["roc_50_100"]],
                "roc_0_50": [dict_res["roc_0_50"]],
                "roc_0_40": [dict_res["roc_0_40"]],
                "roc_0_30": [dict_res["roc_0_30"]],
                "roc_0_20": [dict_res["roc_0_20"]],
            }
        ).to_csv(f"{folder}/{exp_name}_{i}.csv", index=False)
        mb.write(
            [
                i,
                f"{trn_loss:.6f}",
                f"{val_loss:.6f}",
                f"{metric_:.6f}",
                f"{dict_res['roc_all']:.6f}",
                f"{dict_res['roc_50_100']:.6f}",
                f"{dict_res['roc_0_50']:.6f}",
                f"{dict_res['roc_0_40']:.6f}",
                f"{dict_res['roc_0_30']:.6f}",
                f"{dict_res['roc_0_20']:.6f}",
            ],
            table=True,
        )
    print("Training done")
    # loading the best checkpoint

In [ ]:

def scale_data(data):
     return (data-np.min(data))/(np.max(data)-np.min(data))

class DataV0():
    """
    dataset = Dataset(data_type, df)

    img, y = dataset[i]
      img (np.float32): 2 x 360 x 128
      y (np.float32): label 0 or 1
    """
    def __init__(self, df, freq_tfms=False):
        self.df = df
        self.freq_tfms = freq_tfms
        

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        """
        i (int): get ith data
        """
        r = self.df.iloc[i]
        y = np.float32(r.target)
        filename=r.id
        file_id = Path(r.id).stem
        img = np.empty((2, 360, 256), dtype=np.float32)
        with h5py.File(filename, 'r') as f:
            g = f[file_id]

            for ch, s in enumerate(['H1', 'L1']):
                a = g[s]['SFTs'][:, :4096] * 1e22  # Fourier coefficient complex64

                p = a.real**2 + a.imag**2  # power
                p /= np.mean(p)  # normalize
                p = np.mean(p.reshape(360, 256, 16), axis=2)  # compress 4096 -> 128
                #print(p.min(), p.max())
                #p = scale_data(p)
                img[ch] = p
        if self.freq_tfms:
            if np.random.rand()>0.5:
                img = self.freq_tfms(img)

        return img, y.astype('int')

In [ ]:
class CFG:
    bs = 64
    nw = 4
    model_name = "convnext_xlarge_384_in22ft1k"
    lr = 1e-4
    wd = 1e-4
    epoch = 5
    warmup_pct = 0.1
    exp_name = "EXP_20_DATA_V10_V11_V12_NEG_13_14_16_17_360_128_V_16"
    num_classes = 2
    dropout_rate = 0.3
    folder = "EXP_20_00"
    split_voldf = Path("../data/SPLITS/V_16")
    mixup=False

In [ ]:
dforig = pd.read_csv('../data/train_labels.csv')
dforig.columns = ['fn', 'target']
dforig['fn'] = dforig['fn'].apply(lambda x: Path('../data/train')/f'{x}.hdf5')
dforig.columns = ['id', 'target']
#dforig = dforig[dforig.target >= 0].reset_index(drop=True)
trn_df = pd.read_csv(CFG.split_voldf/'trn_df.csv')
val_df = pd.read_csv(CFG.split_voldf/'val_df.csv')
trn_df = pd.concat([dforig.query('target==0').reset_index(drop=True), trn_df]).sample(frac=1)
trn_df.shape,  val_df.shape

In [ ]:
dataset =  DataV0(trn_df, RollTimeFreq())


for i in range(10):
    img, y = dataset[np.random.randint(0, len(dataset))]
    #img, y = dataset[i]
    plt.figure(figsize=(8, 8))
    plt.title('Spectrogram')
    plt.xlabel('time')
    plt.ylabel('frequency')
    plt.imshow(np.concatenate([img[0], img[1]], 1))  # zooming in for dataset[10]
    plt.colorbar()
    plt.show()
    print(y)
    plt.pause(0.1)

    

In [ ]:
gt = []
preds = []
fold = 0

# Train - val split
trn_ds = DataV0(trn_df, RollTimeFreq())
vld_ds = DataV0(val_df)

trn_dl = DataLoader(
    trn_ds,
    batch_size=CFG.bs,
    shuffle=True,
    num_workers=CFG.nw,
    pin_memory=True,
    drop_last=True,
)
vld_dl = DataLoader(
    vld_ds,
    batch_size=CFG.bs,
    shuffle=False,
    num_workers=CFG.nw,
    pin_memory=True,
)


custom_model = create_model(
    CFG.model_name,
    pretrained=True,
    num_classes=CFG.num_classes,
    in_chans=2,
)
opt = torch.optim.AdamW(custom_model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
loss_func = nn.CrossEntropyLoss()
warmup_steps = int(len(trn_dl) * int(CFG.warmup_pct * CFG.epoch))
total_steps = int(len(trn_dl) * CFG.epoch)
sched = get_linear_schedule_with_warmup(
    opt, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

fit_mixup(
    epochs=CFG.epoch,
    model=custom_model,
    train_dl=trn_dl,
    valid_dl=vld_dl,
    loss_fn=loss_func,
    opt=opt,
    val_df = val_df,
    metric=custom_auc_score,
    folder=CFG.folder,
    exp_name=f"{CFG.exp_name}_{fold}",
    device="cuda:0",
    sched=sched,
)
del custom_model
del trn_dl
del vld_dl
import gc

gc.collect()

In [ ]:

epoch	train_loss	valid_loss	val_metric	roc_all	roc_50_100	roc_0_50	roc_0_40	roc_0_30	roc_0_20
0	0.531424	0.487771	0.795653	0.795653	0.963788	0.627518	0.609552	0.611548	0.652328
1	0.496563	0.469109	0.819678	0.819678	0.984185	0.655171	0.615622	0.598838	0.628265
2	0.484414	0.448635	0.826526	0.826526	0.986755	0.666296	0.628585	0.610908	0.626292
3	0.471053	0.470826	0.832730	0.832730	0.990788	0.674672	0.631519	0.608003	0.623880
4	0.457353	0.433260	0.835977	0.835977	0.991256	0.680698	0.639376	0.615618	0.631231

In [ ]:
class TimmFeatureExtractor(nn.Module):
    def __init__(self, model_name, n_class, weights):
        super().__init__()
        self.md = create_model(
            model_name,
            pretrained=False,
            num_classes=n_class,
            in_chans=2,
        )
        print(f"loading: {weights}")
        self.md.load_state_dict(torch.load(weights))

    def forward(self, x):
        with torch.no_grad():
            out = F.softmax(self.md(x))
        return out
    
    
class AverageModel(nn.Module):
    def __init__(self, model_list):
        super().__init__()
        self.model_list = model_list
        
    def forward(self, x):
        res = []
        for md in self.model_list:
            res.append(md(x).detach().cpu())
        return torch.stack(res).mean(0)

In [ ]:
epoch = 4
md_wts = [f"{CFG.folder}/{CFG.exp_name}_0_{epoch}.pth"]
md_wts

In [ ]:
md = AverageModel(
    [
        TimmFeatureExtractor(CFG.model_name, CFG.num_classes, md_wts[0]).cuda().eval(),
    ]
)


In [ ]:
submit = pd.read_csv('../data/sample_submission.csv')
submit.columns = ['fn', 'target']
submit['fn'] = submit['fn'].apply(lambda x: Path('../data/test')/f'{x}.hdf5')
submit.columns = ['id', 'target']
tst_ds = DataV0(submit)
tst_dl = DataLoader(
    tst_ds,
    batch_size=64,
    shuffle=False,
    num_workers=CFG.nw,
    pin_memory=True,
)

res = []
with torch.no_grad():
    for x, y in tqdm(tst_dl):
        out = md(x.cuda())
        res.append(out[:, 1].numpy())



submit['target'] = np.concatenate(res)
submit.columns = ['fn', 'target']
submit['fn'] = submit['fn'].apply(lambda x: x.stem)
submit.columns = ['id', 'target']
os.makedirs('../data/SUBS', exist_ok=True)
submit.to_csv(f'../data/SUBS/{CFG.model_name}_{CFG.exp_name}.csv', index=False)


In [ ]:
submit['target'].plot.hist()